In [ ]:
#Modules to import

#For Web Scraping
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

#For data manipulation
import pandas as pd
import re
from datetime import date

#To slow down the program when scraping. 
import time

In [ ]:
PATH = 'Path\chromedriver.exe'

In [ ]:
price = []
details = []
address = []

#Can only run one state at a time. 
cities = ["Plano", "Richardson", "Garland", "Sachse", "Wylie", "Rowlett", "Addison"]
state = "Tx"

#Initial page request.
options = webdriver.ChromeOptions()
options.add_argument('javascript.enabled')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches",["enable-automation"])
options.add_argument("--incognito")
options.add_argument("--disable-site-isolation-trials")
driver = webdriver.Chrome(executable_path=PATH, options=options)
driver.maximize_window()

#Scraping realtor.com
driver.get('https://www.realtor.com')

x = 0
for city in cities:
    if x == 0:
        t =  30
    else:
        t = 10
    element = driver.find_element_by_xpath('//input[@data-testid="input-element"]')
    try:
        element1 = driver.find_element_by_xpath('//button[@data-testid="cancel"]').click()
    except:
        pass
    time.sleep(5)
    element.send_keys(city + ", " + state)
    time.sleep(6)
    element.send_keys(Keys.RETURN)
    
    time.sleep(t)
    page_num = []
    last_page = 0
    num_pages = []

    num_pages = driver.find_elements_by_xpath('//div[@data-testid="pagination"]//a[@class="item btn "]')
    if len(num_pages) > 0:
        for num in num_pages[:len(num_pages) - 1]:
            page_num.append(int(num.text))
        last_page = max(page_num)
    else:
        last_page = 1

    for page in range(last_page):

        time.sleep(10)
        house_price = []
        house_details = []
        house_address = []
        
        
    
        house_price = driver.find_elements_by_xpath('//li[@data-testid="result-card"]//div[@data-label="property-card"]//div[@data-label="pc-price-wrapper"]')
        house_details = driver.find_elements_by_xpath('//li[@data-testid="result-card"]//div[@data-label="property-card"]//div[@data-testid="property-meta-container"]')
        house_address = driver.find_elements_by_xpath('//li[@data-testid="result-card"]//div[@data-label="property-card"]//div[@data-label="pc-address"]')
    
        for item in range(len(house_price) - 1):
            price.append(house_price[item].text)
            details.append(house_details[item].text)
            address.append(house_address[item].text)
        
        if (page + 1) < last_page:
            next_page = driver.find_elements_by_xpath('//div[@data-testid="pagination"]//a[@class="item btn "]')[-1].click()
        
        x += 1
        time.sleep(10)

driver.close()

In [ ]:
#Functions to clean scrape data.
def house_details(str, detail):
    found = re.search(r'([0-9]*\.?,?[0-9]+)(' + detail + ')' , str)
    if found:
        if found.group(2) == 'acre lot':
            return float(found.group(1).replace(",", "")) * 43560
        else:
            return float(found.group(1).replace(",", ""))
    else:
        return None

def prices(str):
    found = re.search(r'(\$)([0-9]*,?[0-9]*,?[0-9]*)', str)
    if found:
        return int(found.group(2).replace(",", ""))
    return 0

def address_breakdown(address, part):
    ab = re.search(r'([0-9 A-Z a-z]+ [A-Z a-z 0-9]+),\n([A-Z, a-z]+), TX (7[0-9]{4})', address)
    apt = re.search(r'(Apt | Lot | Unit )', address)
    if apt:
        apt = "Y"
    else:
        apt = "N"
    if ab:
        breakdown = [ab.group(1), ab.group(2), ab.group(3), apt]
    else:
        return "No match found"
    return breakdown[part]

In [ ]:
houses = []
for entry in range(len(price)):
    houses.append([price[entry], details[entry], address[entry]])
data = pd.DataFrame(houses, columns=['Price', 'House Details', 'Address'])
today = date.today().strftime("%Y-%m-%d")

In [ ]:
data['Date'] = today

In [ ]:
data['Price'] = data['Price'].apply(prices).astype('int64')

In [ ]:
data['Beds'] = data['House Details'].apply(house_details, args=("bed",)).astype('float64')

In [ ]:
data['Baths'] = data['House Details'].apply(house_details, args=("bath",)).astype('float64')

In [ ]:
data['House sqft'] = data['House Details'].apply(house_details, args=("sqft",)).astype('float64')

In [ ]:
data['Lot sqft'] = data['House Details'].apply(house_details, args=("[a-z]+ lot",)).astype('float64')

In [ ]:
data['Street'] = data['Address'].apply(address_breakdown, args=(0,))

In [ ]:
data['City'] = data['Address'].apply(address_breakdown, args=(1,))

In [ ]:
data['State'] = State

In [ ]:
data['Zipcode'] = data['Address'].apply(address_breakdown, args=(2,))

In [ ]:
data['Apt (Y/N)'] = data['Address'].apply(address_breakdown, args=(3,))

In [ ]:
data = data[['Date', 'Price', 'Street', 'City', 'State', 'Zipcode', 'Apt (Y/N)', 'Beds', 'Baths', 'House sqft', 'Lot sqft']]

In [ ]:
data['Full Address'] = data['Street'] + ', ' + data[['City', 'State', 'Zipcode']] .agg(' '.join, axis=1)data = data[(data['Street'] != "No match found") & (data.City.isin(cities))]

In [ ]:
data = data[(data['Price'] > 0 )]

In [ ]:
data.fillna({'Beds':'NULL', 'Baths':'NULL', 'House sqft':'NULL', 'Lot sqft':'NULL'}, inplace=True)

In [ ]:
#Saves it to an excel file. 
data.to_excel(r"Path\\" + today + " Housing Data.xlsx",
             sheet_name='Housing Data', index=False)